### 一. 每日百万交易的支付系统，如何设置JVM堆内存大小？
如果每日百万交易，但从JVM的角度来看，就是每天会在JVM中会频繁的创建和销毁100万个支付订单，所以这里就牵扯到如下几个核心的问题。
#### 1. 我们的支付系统需要部署多少台机器？  
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;假设每天100万个支付订单，每天高峰期大概是几个小时，用100万平均分配到几个小时里，那么大概是每秒100笔订单左右; 假设我们的支付系统部署了3台机器，每台机器实际上每秒大概处理30笔订单。

#### 2. 每台机器需要多大的内存空间？
首先可以大概计算以下订单对象的大小([如何计算对象大小](../深入理解Java虚拟机/13-%5B%E9%99%84%E5%BD%95%5DJVM%E7%9A%84%E5%B0%8F%E7%9F%A5%E8%AF%86.ipynb)); 一般来说，比如支付订单这种核心类，就按20个实例变量来计算，然后一般大概一个对象也就在几百字节的样子
我们算他大一点好了，就算一个支付订单对象占据500字节的内存空间，不到1kb。


#### 3. 每台机器上启动的JVM需要分配多大的堆内存空间？
 之前说过，假设有3台机器，每秒钟处理30笔支付订单的请求，大概占据的内存空间是30 * 500字节 = 15000字节，大概其实也就15kb而已。其实是非常非常小的。真实的支付系统线上运行，肯定每秒会创建大量其他的对象，但是我们结合这个访问压力以及核心对象的内存占据，大致可以来估算一下整个支付系统每秒钟大致会占据多少内存空间。

其实如果你要估算的话，可以把之前的计算结果扩大10倍~20倍。也就是说，每秒钟除了在内存里创建支付订单对象，还会创建其他数十种对象。

那么每秒钟创建出来的被栈内存的局部变量引用的对象大致占据的内存空间就在几百KB~1MB之间。
 
#### 4. 如果堆内存设置过小会发生什么? 
如果堆内存设置太小, 导致新生代所占比例就更小, 如果每秒反复生成30个订单, 而订单处理速度又很慢, 导致频繁minorGC, 直到新生代无法保存这些订单时, 一部分订单对象会直接进入老年代, 导致老年代GC, 发生Stop world, 系统卡顿;直到老年代也无法承载这些订单对象后, 系统内存溢出
